# Part 1: Converting a DNA sequence into an amino acid sequence

In [110]:
codon_chart = {'AUG': 'M', 'UAA': 'STOP', 'UAG': 'STOP', 'UGA': 'STOP', 
               'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
               'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
              'UAU': 'Y', 'UAC': 'Y', 'UGU': 'C', 'UGC': 'C', 'UGG': 'W',
              'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
              'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
              'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
              'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
              'AUU': 'I', 'AUC': 'I', 'AUA': 'I',
              'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
              'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
              'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
              'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
              'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
              'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
              'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}

dna_seq = input('Please type your DNA sequence:')
def dnaseq_aminoacidseq(dna_seq):
    rna_seq = dna_seq.replace('T', 'U') # Convert to mRNA seq
    amino_acids = []
    for i in range(0, len(rna_seq), 3): # Range from 1st to last index, in steps of 3 (bc every 3 nucleotides is 1 codon)
        codon = rna_seq[i:i+3] # 1 codon is made of 3 nucleotides
        if codon in codon_chart:
            amino_acid = codon_chart[codon]
            if amino_acid == 'STOP':
                break
            amino_acids.append(amino_acid)
    amino_acids_string = ''.join([str(element) for element in amino_acids]) # Converting list to string
    return amino_acids_string

amino_acid_seq = dnaseq_aminoacidseq(dna_seq)
print(f"Here is your amino acid sequence: {amino_acid_seq}")

# Can use .transcribe() and .translate()

Please type your DNA sequence: ATGGTGCATCTGACCCCGGAAGAAAAAAGCGCGGTGACCGCGCTGTGGGGCAAAGTGAACGTGGATGAAGTGGGCGGCGAAGCGCTGGGCCGCCTGCTGGTGGTGTATCCGTGGACCCAGCGCTTTTTTGAAAGCTTTGGCGATCTGAGCACCCCGGATGCGGTGATGGGCAACCCGAAAGTGAAAGCGCATGGCAAAAAAGTGCTGGGCGCGTTTAGCGATGGCCTGGCGCATCTGGATAACCTGAAAGGCACCTTTGCGACCCTGAGCGAACTGCATTGCGATAAACTGCATGTGGATCCGGAAAACTTTCGCCTGCTGGGCAACGTGCTGGTGTGCGTGCTGGCGCATCATTTTGGCAAAGAATTTACCCCGCCGGTGCAGGCGGCGTATCAGAAAGTGGTGGCGGGCGTGGCGAACGCGCTGGCGCATAAATATCAT


Here is your amino acid sequence: MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH


##### A function to convert a DNA sequence into an amino acid sequence is defined, where the input (DNA sequence) is first converted to an mRNA sequence and subsequently to an amino acid acid sequence. A dictionary for the codon chart is defined, where each mRNA codon corresponds to an amino acid.

# Part 2: Protein BLAST

In [106]:
from Bio import Blast # BLAST module from Biopython

result_stream = Blast.qblast('blastp', 'pdb', amino_acid_seq)
# qblast passes sequence data as the main parameter
# Internal programme: blastp for protein. Otherwise, blastn is for nucleotides
# Database: pdb to access pdb files only
# Results are given in XML format by default

with open("my_blast.xml", "wb") as out_stream:
    out_stream.write(result_stream.read())
# wb: File is opened in write-binary mode
# Results are read and written to a file
result_stream = open("my_blast.xml", "rb")
# rb: File is opened in read-binary mode

blast_record = Blast.read(result_stream)
# File is opened and read in BLAST

print(blast_record[0][0], # Access the first record (though only one query is input in this example) and first hit (alignment)
# There are multiple hits in 1 record
      '\n\n', blast_record[0].target.name) # Sequence name/ identifier 

Query : Query_710761 Length: 147 Strand: Plus
        unnamed protein product
Target: pdb|1DXT|B Length: 147 Strand: Plus
        Chain B, HEMOGLOBIN (DEOXY) (BETA CHAIN) [Homo sapiens] >pdb|1DXT|D
        Chain D, HEMOGLOBIN (DEOXY) (BETA CHAIN) [Homo sapiens] >pdb|6KYE|B
        Chain B, Hemoglobin subunit beta [Homo sapiens] >pdb|6KYE|D Chain D,
        Hemoglobin subunit beta [Homo sapiens] >pdb|6KYE|F Chain F, Hemoglobin
        subunit beta [Homo sapiens] >pdb|6KYE|H Chain H, Hemoglobin subunit beta
        [Homo sapiens] >pdb|6KYE|J Chain J, Hemoglobin subunit beta [Homo
        sapiens] >pdb|6KYE|L Chain L, Hemoglobin subunit beta [Homo sapiens]
        >pdb|7CUE|B Chain B, Hemoglobin subunit beta [Homo sapiens] >pdb|7CUE|D
        Chain D, Hemoglobin subunit beta [Homo sapiens] >pdb|7UD7|B Chain B,
        Hemoglobin subunit beta [Homo sapiens] >pdb|7UD7|D Chain D, Hemoglobin
        subunit beta [Homo sapiens] >pdb|7UD8|B Chain B, Hemoglobin subunit beta
        [Homo sapiens

##### The Biopython library is used to perform a BLAST (Basic Local Alignment Search Tool) search for the protein sequence in part 1 against the PDB (Protein Data Bank) database. The first hit of the BLAST record is printed.

# Part 3: Extracting information on secondary structure assignment using DSSP

In [112]:
from biopandas.pdb import PandasPdb
from Bio.PDB import PDBParser, DSSP

pdbid = blast_record[0].target.name.split('_')[0]
ppdb = PandasPdb().fetch_pdb(pdbid) # because id also specifies the chain but pdb cannot read it so just taking the id of the whole protein
file_path = f'{pdbid}.pdb'
ppdb.to_pdb(path = file_path)

# Load the PDB file
parser = PDBParser()
structure = parser.get_structure(pdbid, file_path)

# Creating DSSP object from PDB file
dssp = DSSP(structure[0], file_path) # Each PDB file can have multiple models, so [0] accesses the 1st model

# Access data from DSSP
for key in dssp.keys(): # Each key represents 1 residue
    residue = dssp[key] # Each residue contains data of multiple characteristics of the residue
    residue_id = residue[0] # Accesses 1st element of the residue data (residue identity by number) 
    secondary_structure = residue[2]  # Accesses secondary structure assignment in residue data
    print(f"Residue {residue_id} has secondary structure: {secondary_structure}")

/Users/gabrielle/miniconda3/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4903.
  warnings.warn(
/Users/gabrielle/miniconda3/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4946.
  warnings.warn(
/Users/gabrielle/miniconda3/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 4989.
  warnings.warn(
/Users/gabrielle/miniconda3/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 5032.
  warnings.warn(
/Users/gabrielle/miniconda3/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5075.
  warnings.warn(
/Users/gabrielle/miniconda3/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstruction

Residue 1 has secondary structure: -
Residue 2 has secondary structure: -
Residue 3 has secondary structure: -
Residue 4 has secondary structure: H
Residue 5 has secondary structure: H
Residue 6 has secondary structure: H
Residue 7 has secondary structure: H
Residue 8 has secondary structure: H
Residue 9 has secondary structure: H
Residue 10 has secondary structure: H
Residue 11 has secondary structure: H
Residue 12 has secondary structure: H
Residue 13 has secondary structure: H
Residue 14 has secondary structure: H
Residue 15 has secondary structure: H
Residue 16 has secondary structure: H
Residue 17 has secondary structure: H
Residue 18 has secondary structure: G
Residue 19 has secondary structure: G
Residue 20 has secondary structure: G
Residue 21 has secondary structure: H
Residue 22 has secondary structure: H
Residue 23 has secondary structure: H
Residue 24 has secondary structure: H
Residue 25 has secondary structure: H
Residue 26 has secondary structure: H
Residue 27 has second

##### The DSSP (Dictionary of Secondary Structures in Proteins) is a database that contains information on the secondary structure of proteins in the PDB. 

##### A standard assignment is used, where each secondary structure type is assigned a letter:
|Letter|Structure type|
|:---|:---|
|H|Alpha helix (4-12)|
|B|Isolated beta-bridge residue|
|E|Strand|
|G|3-10 helix|
|I|Pi helix|
|T|Turn|
|S|Bend|
|-|None|

# Part 4: Visualisation using NGLView

In [113]:
import nglview as nv
# Creating a dictionary of colours assigned to each sec structure type
colour_dict = {'H': 'purple',
             'B': 'green',
             'E': 'orange',
             'G': 'pink',
             'I': 'blue',
             'T': 'red',
             'S': 'gray',
              '-': 'white'}

# Creating visualisation
view = nv.show_file(file_path)
view.add_representation('ribbon')

for key in dssp.keys(): # Each key represents 1 residue
    residue = dssp[key] # Each residue contains data of multiple characteristics of the residue
    residue_number = residue[0] # Accesses 1st element of the residue data (residue identity by number) 
    secondary_structure = residue[2]  # Accesses secondary structure assignment in residue data
    colour = colour_dict[secondary_structure] # Access corresponding colour to sec structure assignment
    view.update_representation(sele="residue" + str(residue_number), color=colour) # Tells python to access the residue, and then select the residue number
view

NGLWidget()

##### The structure of the protein is visualised using NGLView, where each outline colour represents one type of secondary structure.